In [1]:
import numpy as np

### Introduction
Model of electric engine. 
1000 - 10000 RPMs.

Differential equation describing the system.

$$J\frac{d\omega}{dt} = M_e - M_0 - M_{LOAD}$$
where:
- $M_e$: electromagnetic moment
- $M_0$: braking moment
- $M_{LOAD}$: load moment
- $I$: moment of inertia

Transformation to differential form using the Eulerian method.
$$I\frac{d\omega(t+\Delta t) - \omega (t)}{\Delta t} = M_e(t)-M_0-M_{LOAD}(t)$$
$$\omega (t+\Delta t) = \omega(t) + \frac{\Delta t}{I}(M_e(t)- M_0-M_{LOAD}(t))$$

What about $M_e(t)$?
$$U_{PI}(t) = Kp \cdot e(t) + Ki \sum^t_{k=0}e(k)\Delta t$$
$$M_e(t) = U_{PI} * const$$
$$e(t) = \omega_{ref} - \omega(t)$$

### Defined Parameters

In [ ]:
# Parameters of simulation
referencedRevolutionsPerMinute = 2000

# Simulation time parameters
timeOfSimulation = 30
timeOfSample = 0.1

# Parameters of crankshaft
brakingMoment = 0.5
loadMoment = 0.5
momentOfInertia = 1.2
constantOfElectromagneticMoment = 0.1 #! dostosowac

# Parameters of PI regulator
Kp = 0.1
Ki = 0.1

# Constraints
Umax = 24
Umin = 0
maximumRevolutionsPerMinute = referencedRevolutionsPerMinute * 1.5 #! Check
maximumElectromagneticMoment = maximumRevolutionsPerMinute / 1500 #! Check

# Lists of measured values
# TODO: Think about initalizing first elements here in list
timeOfSimulationList = [0.0]
loadMomentList = [] 
electromagneticMoment = []
adjustmentErrors = [referencedRevolutionsPerMinute]

### Calculations

In [ ]:
def calculateNumberOfSteps(timeOfSimulation: int, timeOfSample: float) -> int:
    """ This function calculates number of iterations for simulation of process

        @Parameters:
        - timeOfSimulation (int): total time of simulation in seconds
        - timeOfSample (float): time at which we repeat the measurement in seconds

        @Return:
        - int: number of iterations
    """
    return int(timeOfSimulation / timeOfSample) + 1

In [ ]:
def calculateAdjustmentError(referencedRevolutionsPerMinute: float, currentRevolutionsPerMinute: float) -> float:
    # TODO Floats or Integers???
    """ This function calculates adjustment error which is difference between referenced value and current one

        @Parameters:
        - referencedRevolutionsPerMinute (float): set value to be obtained by regulator
        - currentRevolutionsPerMinute (float): current value

        @Return:
        - float: error
    """
    return referencedRevolutionsPerMinute - currentRevolutionsPerMinute

In [ ]:
def calculateVoltageOfRegulator(errorList : list[float], iteration: int) -> float:
    """ This funtion calculates current voltage of regulator

        @Parameters:
        - errorList (list[float]): list of errors at the moment and before
        - iteration (int): information about current simulation iteration

        @Return
        - float: current voltage of regulator
    """
    return Kp * errorList[iteration] + Ki * sum(errorList) * timeOfSample

In [ ]:
def calculateElectromagneticMoment(constant : float, currentVoltage : float) -> float:
    """ This function calculates current electromagnetic moment based on voltage of regulator

        @Parameters:
        - constant (float): used to scale moment 
        - currentVoltageOfRegulator (float): voltage of regulator at the moment

        @Return
        - float: current electromagnetic moment
    """
    return constant * currentVoltage

In [ ]:
def calculateNormalizedVoltage(voltgeOfRegulator : float) -> float:
    """ This function calculates normalized voltage based on predefined constraints <0;24> [V]

    @Parameters:
    - voltageOfRegulator (float): current voltage of regulator

    @Returns:
    - float: normalized voltage used to create electromagnetic moment
    """
    return max(Umin, min(Umax, voltgeOfRegulator))

### Simulation

In [2]:
# TODO Perform simulation here